In [1]:
import pandas as pd
import numpy as np

import re
from itertools import chain
from collections import Counter

from google.cloud import bigquery
import gspread
from google.oauth2 import service_account

from datetime import datetime, timedelta

import requests
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# InsecureRequestWarning 경고 무시
warnings.simplefilter('ignore', InsecureRequestWarning)

In [2]:
KEY_PATH = ".config/"
servicekey_path = KEY_PATH + "serviceKey.json" ## 빅쿼리 외 다른 API 활용 위해
bigquerykey_path = KEY_PATH + "mido-project-426906-31b49963ac97.json"
gskey_path = KEY_PATH + "mido-project-426906-e75771d2b77a.json"

warnings.filterwarnings("ignore")

In [3]:
# BigQuery 클라이언트 생성 함수
def create_bigquery_client(key_path):
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    return client

In [4]:
# google sheets 클라이언트 생성 함수
def create_sheets_client(key_path):
    # Google Sheets API 인증
    scope = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive"
    ]
    credentials = service_account.Credentials.from_service_account_file(key_path, scopes=scope)
    client = gspread.authorize(credentials)
    return client

In [5]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    # BigQuery 클라이언트 객체 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [6]:
def get_dataframe_from_bigquery(dataset_id, table_id, key_path):
    # BigQuery 클라이언트 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 테이블 데이터를 DataFrame으로 변환
    df = client.list_rows(table_ref).to_dataframe()

    return df

In [7]:
# 오늘 날짜
today = datetime.today()#.strftime('%Y%m%d')

# 어제 날짜 계산
ytday = datetime.today() - timedelta(days=1)

# 만약 어제, 오늘이 토요일(5) 또는 일요일(6)이라면, 그 전주 금요일로 변경
if ytday.weekday() == 5:  # 토요일
    ytday -= timedelta(days=1)
elif ytday.weekday() == 6:  # 일요일
    ytday -= timedelta(days=2)
if today.weekday() == 5:  # 토요일
    today -= timedelta(days=1)
elif today.weekday() == 6:  # 일요일
    today -= timedelta(days=2)

# 'YYYYMMDD' 형식으로 변환
ytday = ytday.strftime('%Y%m%d')
today = today.strftime('%Y%m%d')

#### 법정동코드

In [8]:
dist_code = pd.read_csv('C:\py_src\midoproject\data/법정동코드 전체자료.csv',encoding='cp949')

In [9]:
dist_code_list = []
for i in range(dist_code['법정동명'].str.split().apply(len).max()):
    dist_code_temp = dist_code[dist_code['법정동명'].str.split().str[i].str[-1].isin(['읍','면','동','리','가','로']).dropna()]
    dist_code_list.append([dist_code_temp['법정동명'].str.split().str[i].dropna().unique()])

emd_code_list = list(chain(*chain(*dist_code_list)))

#### 종합쇼핑몰 납품상세내역

In [10]:
all_shop_df = get_dataframe_from_bigquery('g2b', 'shop_detail_df_all', bigquerykey_path)

In [11]:
all_shop_df['납품요구변경차수'] = all_shop_df['납품요구변경차수'].astype(int)
all_shop_df = all_shop_df.loc[all_shop_df.groupby(['납품요구번호', '물품순번'])['납품요구변경차수'].idxmax()]
all_shop_df = all_shop_df.sort_values(['납품요구접수일자'],ascending=False).reset_index(drop=True)

In [12]:
all_shop_df_fin = all_shop_df[['납품요구접수일자', '수요기관명', '납품요구건명', '업체명', '단가', '단위', '수량', '금액', '품목', '수요기관지역명', '납품요구지청명','업체기업구분명','우수제품여부']]

In [13]:
# 컬럼명을 한글로 변경
column_mapping = {
    '납품요구접수일자' : 'dlvrReqRcptDate',
    '품목' : 'prdctIdntNoNm',
    '단가' : 'prdctUprc',
    '단위' : 'prdctUnit',
    '수량' : 'prdctQty',
    '금액' : 'prdctAmt',
    '우수제품여부' : 'exclcProdctYn',
    '수요기관명' : 'dminsttNm',
    '수요기관지역명' : 'dminsttRgnNm',
    '업체명' : 'corpNm',
    '납품요구건명' : 'dlvrReqNm',
    '업체기업구분명' : 'corpEntrprsDivNmNm',
    '납품요구지청명' : 'brnofceNm'
}

all_shop_df_fin.rename(columns=column_mapping, inplace=True)

In [14]:
all_shop_df_fin[all_shop_df_fin['dlvrReqNm'].str.contains('축구장')]['prdctIdntNoNm'].str.split(',').str[-1].unique()

array([' 충전재/충격흡수패드', ' t55mm', ' 충전재', ' W2000×t35mm', ' t45mm', ' 이중',
       ' t35mm', ' t20mm', ' W4000×t35mm', ' W4000×t55mm', ' t19mm',
       ' W3800×t55mm', ' W4000×t45mm', ' 인조잔디하부', ' 충격흡수패드',
       ' W4000×55mm', ' 무충진', ' W3700×t13mm', ' SEBS황토칩', ' SEBS컬러칩',
       ' EPDM칩흑색', ' W4000×T55mm', ' SEBS칩', ' SEBS', ' SEBS압출칩',
       ' SEBS녹색칩', ' W4000×35mm', ' 규사', ' SEBS압출칩KONYFILL/컬러',
       ' SEBS압출칩(컬러)', ' W4100×55mm', ' 코르크칩갈색',
       ' 익스트림모노필라멘트/EPDM BLACK 충진칩 흑색', ' 컬러', ' W4000×57mm',
       ' W3650×35mm', ' SEBS압출칩KONYFILL컬러+올레핀패드삽입', ' 효성에버그린',
       ' 4000×55mm', ' 국산EPDM고무칩(흑색)', ' 익스트림/SEBS사출칩', ' SEBS압출칩+천연칩',
       ' 독일산RPU칩', ' SEBS압출칩KONYFILL(컬러)+칩패드', ' 스페인산SPU칩녹색/갈색',
       ' (국산/SEBS/칼라)', ' SBR칩흑색', ' SEBS사출칩(녹색)', ' Geofill(갈색)',
       ' EPDM고무칩흑색', ' W3660~4000×55mm', ' 갈색', ' GR인증SBR코팅칩/컬러',
       ' SBR흑색'], dtype=object)

In [15]:
# 빅쿼리 적재
save_dataframe_to_bigquery(all_shop_df_fin,'g2b','shop_detail_df_view',bigquerykey_path)

Data inserted into table shop_detail_df_view successfully.


#### 종합쇼핑몰 품목 정보

In [16]:
shop_prod_info_df = get_dataframe_from_bigquery('g2b', 'shop_prod_info', bigquerykey_path)

In [17]:
shop_prod_info_df_temp = shop_prod_info_df[['shopngCntrctNo','shopngCntrctSno','cntrctCorpNm','cntrctMthdNm','prdctClsfcNoNm','dtilPrdctClsfcNoNm','prdctSpecNm','cntrctDeptNm','prodctCertList',
                                          'exclncPrcrmntPrdctYn','masYn','smetprCmptProdctYn','cntrctPrceAmt','prdctDlvrPlceNm','prdctDlvryCndtnNm','prdctSplyRgnNm','dlvrTmlmtDaynum']]
shop_prod_info_df_temp = shop_prod_info_df_temp[(shop_prod_info_df_temp['prdctClsfcNoNm']=='인조잔디')].reset_index(drop=True)

In [18]:
# 컬럼명을 한글로 변경
column_mapping = {
    'shopngCntrctNo': '쇼핑계약번호',
    'shopngCntrctSno': '쇼핑계약순번',
    'cntrctCorpNm': '업체명',
    'cntrctMthdNm': '계약방법',
    'prdctClsfcNoNm': '물품명',
    'dtilPrdctClsfcNoNm': '세부물품명',
    'prdctSpecNm': '물품식별명',
    'cntrctDeptNm': '계약부서명',
    'prodctCertList': '인증목록',
    'exclncPrcrmntPrdctYn': '우수조달물품여부',
    'masYn': '다수공급경쟁자여부',
    'smetprCmptProdctYn': '중소기업자간경쟁제품여부',
    'cntrctPrceAmt': '금액',
    'prdctDlvrPlceNm': '납품장소명',
    'prdctDlvryCndtnNm': '인도조건명',
    'prdctSplyRgnNm': '공급지역명',
    'dlvrTmlmtDaynum': '납품기한'
}

shop_prod_info_df_temp.rename(columns=column_mapping, inplace=True)

In [19]:
# 제품 추출
shop_prod_info_df_temp['제품'] = shop_prod_info_df_temp['물품식별명'].apply(lambda x: re.findall('t19mm|t20mm|t35mm|t45mm|t55mm',x)).str[0]

In [20]:
# 전처리
shop_prod_info_df_temp['잔디구조이중여부'] = ''
shop_prod_info_df_temp['잔디구조이중여부'] = np.where(shop_prod_info_df_temp['물품식별명'].str.split(',').str[4:].apply(''.join).str.contains('이중'),'Y','N')

shop_prod_info_df_temp['충전재여부'] = ''
shop_prod_info_df_temp['충전재여부'] = np.where(shop_prod_info_df_temp['물품식별명'].str.split(',').str[4:].apply(''.join).str.contains('충전재'),'Y','N')

shop_prod_info_df_temp['충격흡수패드여부'] = ''
shop_prod_info_df_temp['충격흡수패드여부'] = np.where(shop_prod_info_df_temp['물품식별명'].str.split(',').str[4:].apply(''.join).str.contains('충격흡수패드여부'),'Y','N')

In [21]:
# 인증목록별 melt
shop_prod_info_df_temp['인증목록'] = shop_prod_info_df_temp['인증목록'].str.split(',')
shop_prod_info_df_temp = shop_prod_info_df_temp.explode('인증목록').reset_index(drop=True)
shop_prod_info_df_temp['인증목록'] = shop_prod_info_df_temp['인증목록'].str.split('(').str[0]

In [22]:
# 55mm 우수제품정보
turf_standard_55 = pd.read_csv('C:\py_src\midoproject\data/turf_standard_55.csv', encoding='cp949')

In [23]:
# 전처리
turf_standard_55['제품'] = turf_standard_55['소재'].str.split('/').str[1]
turf_standard_55['제품'] = turf_standard_55['제품'].str.strip()
turf_standard_55['제품'] = 't' + turf_standard_55['제품']

In [24]:
# 범주형 데이터 전처리
turf_standard_55[turf_standard_55.columns[5:-1]] = turf_standard_55[turf_standard_55.columns[5:-1]].applymap(lambda x: int(x.replace('이상','').replace(',','').strip()))

In [25]:
# 종합쇼핑몰 품목 정보, 55mm 우수제품 병합
shop_prod_info_df_fin = pd.merge(shop_prod_info_df_temp, turf_standard_55, how='left', on=['업체명','제품'])

In [26]:
# 에코와 다른 업체 비교
shop_prod_info_df_fin['에코여부'] = '경쟁사'
shop_prod_info_df_fin['에코여부'] = np.where(shop_prod_info_df_fin['업체명'].str.contains('에코그라운드'),'(주)에코그라운드',shop_prod_info_df_fin['에코여부'])

In [27]:
# 최종 데이터셋 정리
shop_prod_info_df_fin.columns = shop_prod_info_df_fin.columns.str.split('(').str[0].str.replace('/','').str.replace(' ','')
shop_prod_info_df_fin = shop_prod_info_df_fin.fillna('')

In [28]:
# Google Sheets 열기
client = create_sheets_client(gskey_path)
spreadsheet = client.open("종합쇼핑몰 품목 정보")
sheet = spreadsheet.worksheet('sheet2')

In [29]:
# Google Sheets에 데이터 업로드
sheet.clear()
sheet.update([shop_prod_info_df_fin.columns.values.tolist()] + shop_prod_info_df_fin.values.tolist())

{'spreadsheetId': '14expk51SfmXKfTrD-Qadqj6CQa9cNQtOBpBQc2Oy_VY',
 'updatedRange': 'sheet2!A1:AH612',
 'updatedRows': 612,
 'updatedColumns': 34,
 'updatedCells': 20808}

In [30]:
# 빅쿼리 적재
save_dataframe_to_bigquery(shop_prod_info_df_fin,'g2b','shop_prod_info_df_view',bigquerykey_path)

ArrowTypeError: Expected bytes, got a 'float' object